*Imports*

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0,'classes')
import grafica as grafica
from ClassNeuronaGral import NeuronaGradiente
from sklearn.model_selection import train_test_split

# Regresión Logística Múltiple

**Ejercicio 10**

El archivo seeds.csv contiene información de granos que pertenecen a tres variedades diferentes de
trigo.

Entrene una neurona no lineal para determinar si una semilla es de “TIPO 2” o no. Utilice el 60% de los ejemplos para entrenar y el 40% para realizar el testeo. Realice 30 ejecuciones independientes de la configuración seleccionada para respaldar sus afirmaciones referidas a la precisión obtenida tanto para el conjunto de entrenamiento como para el de testeo. Utilice un máximo de 200 iteraciones y una cota de error de 1𝑒 − 03.
1. Utilizando ‘sigmoid’ como función de activación.
2. Utilizando ‘tanh’ como función de activación.

In [2]:
# Load dataset
df_semillas = pd.read_csv('data/seeds.csv')

# Numerization
df_semillas.Clase = df_semillas.Clase.map({'Tipo1' : 0, 'Tipo2' : 1, 'Tipo3' : 0})

# Normalization
for c in df_semillas.columns:
  if c != 'Clase':
    mean = df_semillas[c].mean()
    std = df_semillas[c].std()
    df_semillas[c] = (df_semillas[c] - mean) / std

In [3]:
#Split dataframe into train and test
train_set, test_set = train_test_split(df_semillas, test_size=0.4)

#Prepare x and t of train set
examples = np.array(train_set)
columns_name = np.array(train_set.columns.values)

column_to_predict = 7
T_train = examples[:, column_to_predict]
X_train = np.delete(examples, [column_to_predict], 1)

#Prepare x and t of test set
examples = np.array(test_set)
columns_name = np.array(test_set.columns.values)

column_to_predict = 7
T_test = examples[:, column_to_predict]
X_test = np.delete(examples, [column_to_predict], 1)

In [4]:
df_results = pd.DataFrame(columns=[
    'function', 'alpha', 'mean_iterations',
    'train_accuracy_mean',
    'test_accuracy_mean',
    'train_accuracy_count',
    'test_accuracy_count'
    ])
alphas = [0.1, 0.2, 0.3]
max_iterations = 200
bound_error = 1e-03
functions = ['sigmoid', 'tanh']
executions_count = 30
accuracy = lambda y, t: np.sum(y==t) / len(t)
for function in functions:
  for alpha in alphas:
    iterations_sum = 0
    accuracy_train_sum = 0
    accuracy_test_sum = 0
    train_accuracy_count = 0
    test_accuracy_count = 0
    #Repeat the executions
    for r in range(executions_count):
      max_accuracy = -1
      ite_of_max_accuracy = -1
      #Create the neuron and train it
      neuron = NeuronaGradiente(alpha=alpha, n_iter=max_iterations, cotaE=bound_error, FUN=function, random_state=1)
      neuron.fit(X_train, T_train)
      #Calculate the accuracies, depending of the function
      if function == 'tanh':
        """A dif. de la funcion sigmoide,  debo normalizar la entrada
        para poder medir el accuracy,  ya  que la prediccion es entre
        los valores -1 y 1, siendo mi salida esperada entre 0 y 1."""
        predict_train = neuron.predict(X_train)
        predict_test = neuron.predict(X_test)
        for i in range(len(X_train)):
          if predict_train[i] == -1:
            predict_train[i] = 0
        for i in range(len(X_test)):
          if predict_test[i] == -1:
            predict_test[i] = 0
        accuracy_train = accuracy(predict_train, T_train)
        accuracy_test = accuracy(predict_test, T_test)
      else:
        accuracy_train = accuracy(neuron.predict(X_train), T_train)
        accuracy_test = accuracy(neuron.predict(X_test), T_test)
      #Count if it's max accuracy
      if accuracy_train == 1.0:
        train_accuracy_count = train_accuracy_count + 1
      if accuracy_test == 1.0:
        test_accuracy_count = test_accuracy_count + 1
      #Sum the results
      iterations_sum = iterations_sum + len(neuron.errors_)
      accuracy_train_sum = accuracy_train_sum + accuracy_train
      accuracy_test_sum = accuracy_test_sum + accuracy_test
    df_results.loc[len(df_results.index)] = [
        function, alpha,
        (iterations_sum / float(executions_count)),
        accuracy_train_sum / float(executions_count),
        accuracy_test_sum / float(executions_count),
        train_accuracy_count,
        test_accuracy_count
        ]
df_results.head(6)

,function,alpha,mean_iterations,train_accuracy_mean,test_accuracy_mean,train_accuracy_count,test_accuracy_count
0,sigmoid,0.1,200.0,0.968254,1.000000,0,30
1,sigmoid,0.2,200.0,0.976190,1.000000,0,30
2,sigmoid,0.3,200.0,0.992063,1.000000,0,30
3,tanh,0.1,200.0,0.746032,0.761905,0,0
4,tanh,0.2,200.0,0.769841,0.773810,0,0
5,tanh,0.3,200.0,0.833333,0.857143,0,0
